In [541]:
import torch
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [542]:
# Define the Neural Network Model (same as during training)
class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = torch.nn.Linear(input_size, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.layer2 = torch.nn.Linear(128, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.output = torch.nn.Linear(64, output_size)
        self.dropout = torch.nn.Dropout(0.3)
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout(x)
        x = torch.sigmoid(self.output(x))
        return x

In [543]:
# Load the model and scaler
def load_model_and_scaler(model_path, scaler_path):
    # Load the scaler
    scaler = joblib.load(scaler_path)
    
    # Initialize the model
    model = NeuralNetwork(input_size=12, output_size=14)  # Adjust input_size and output_size as needed
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode
    
    return model, scaler

In [544]:
# Function to preprocess input data
def preprocess_input(input_data, scaler):
    input_df = pd.DataFrame([input_data])
    input_scaled = scaler.transform(input_df)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32)
    return input_tensor

In [545]:
# Function to make predictions
def predict_mental_health(input_data, model, scaler, threshold=0.3):
    input_tensor = preprocess_input(input_data, scaler)
    with torch.no_grad():
        output = model(input_tensor).numpy()[0]
    predictions = (output > threshold).astype(int)
    return output, predictions


In [546]:
# Paths to the saved model and scaler
model_path = 'mental_health_model.pth'
scaler_path = 'scaler.pkl'

In [547]:
# Load the model and scaler
model, scaler = load_model_and_scaler(model_path, scaler_path)

C:\Users\anike\AppData\Local\Temp\ipykernel_17604\434327885.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [548]:

input_data = {
    'gender': 1,
    'bmi': 0.2,
    'phq_score': 0.3,
    'gad_score': 0.3,
    'epworth_score': 0.2,
    'depression_severity': 1,
    'anxiety_severity': 2,
    'suicidal': 1,
    'depressiveness': 1,
    'anxiousness': 1,
    'anxiety_diagnosis': 0,
    'sleepiness': 1
}

In [549]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [550]:
class MentalHealthAssistant:
    def __init__(self):
        self.model = T5ForConditionalGeneration.from_pretrained("./mentalhealth-t5")
        self.tokenizer = T5Tokenizer.from_pretrained("./mentalhealth-t5")

        # # Thresholds for urgency classification (adjust as needed)
        # self.thresholds = {
        #     "Major Depressive Disorder (MDD)": 0.,
        #     "Dysthymia": 0.5,
        #     "Seasonal Affective Disorder (SAD)": 0.5,
        #     "Generalized Anxiety Disorder (GAD)": 0.55,
        #     "Panic Disorder": 0.5,
        #     "PTSD": 0.6,
        #     "Insomnia": 0.45,
        #     "Hypersomnia": 0.4,
        #     "Sleep Apnea": 0.4,
        #     "Suicidal Ideation": 0.35,
        #     "High-Risk Crisis": 0.4,
        #     "Chronic Stress": 0.5,
        #     "Adjustment Disorder": 0.45,
        #     "Burnout": 0.4
        # }

        # Thresholds for urgency classification (adjust as needed)
        self.thresholds = {
            "Major Depressive Disorder (MDD)": 0.8,
            "Dysthymia": 0.8,
            "Seasonal Affective Disorder (SAD)": 0.85,
            "Generalized Anxiety Disorder (GAD)": 0.855,
            "Panic Disorder": 0.85,
            "PTSD": 0.86,
            "Insomnia": 0.845,
            "Hypersomnia": 0.84,
            "Sleep Apnea": 0.84,
            "Suicidal Ideation": 0.835,
            "High-Risk Crisis": 0.84,
            "Chronic Stress": 0.85,
            "Adjustment Disorder": 0.845,
            "Burnout": 0.84
        }

    def generate_insights(self, condition, probability):
        """Generate a detailed explanation based on condition and probability."""
        inputs = self.tokenizer(
            f"Explain {condition} with probability {probability:.2f}",
            return_tensors="pt",
            max_length=128,
            truncation=True
        )
        
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )
        
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _classify_severity(self, probability):
        """Classifies severity based on probability."""
        if probability >= 0.7:
            return "Severe"
        elif probability >= 0.5:
            return "Moderate"
        else:
            return "Mild"

    def _determine_urgency(self, condition, probability):
        """Determines urgency level for intervention based on thresholds."""
        threshold = self.thresholds.get(condition, 0.8)  # Default threshold if not specified
        if probability >= threshold:
            return "Immediate medical intervention recommended"
        elif probability >= (threshold * 0.8):  # Adjust for close monitoring cases
            return "Monitor closely, therapy required"
        else:
            return "Routine psychological support suggested"

    def format_report(self, conditions, predictions, probabilities):
        report = []
        for cond, pred, prob in zip(conditions, predictions, probabilities):
            if pred == 1:
                insight = self.generate_insights(cond, prob)
                report.append({
                    "Condition": cond,
                    "Severity": self._classify_severity(prob),
                    "Clinical Insight": insight,
                    "Urgency": self._determine_urgency(cond, prob)
                })
        return pd.DataFrame(report)


        

In [551]:
# Make predictions
probabilities, predictions = predict_mental_health(input_data, model, scaler)
print(type(predictions))

<class 'numpy.ndarray'>


In [552]:
conditions = [
    'Major Depressive Disorder (MDD)', 'Dysthymia', 'Seasonal Affective Disorder (SAD)', 
    'Generalized Anxiety Disorder (GAD)', 'Panic Disorder', 'PTSD', 
    'Insomnia', 'Hypersomnia', 'Sleep Apnea', 
    'Suicidal Ideation', 'High-Risk Crisis', 
    'Chronic Stress', 'Adjustment Disorder', 'Burnout'
]

In [553]:
print("Probabilities:")
for condition, prob in zip(conditions, probabilities):
    print(f"{condition}: {prob:.4f}")

print("\nPredictions (Binary):")
for condition, pred in zip(conditions, predictions):
    print(f"{condition}: {pred}")

Probabilities:
Major Depressive Disorder (MDD): 0.6970
Dysthymia: 0.6840
Seasonal Affective Disorder (SAD): 0.4861
Generalized Anxiety Disorder (GAD): 0.4881
Panic Disorder: 0.4356
PTSD: 0.4523
Insomnia: 0.6210
Hypersomnia: 0.6177
Sleep Apnea: 0.7145
Suicidal Ideation: 0.6959
High-Risk Crisis: 0.6075
Chronic Stress: 0.5933
Adjustment Disorder: 0.4608
Burnout: 0.4605

Predictions (Binary):
Major Depressive Disorder (MDD): 1
Dysthymia: 1
Seasonal Affective Disorder (SAD): 1
Generalized Anxiety Disorder (GAD): 1
Panic Disorder: 1
PTSD: 1
Insomnia: 1
Hypersomnia: 1
Sleep Apnea: 1
Suicidal Ideation: 1
High-Risk Crisis: 1
Chronic Stress: 1
Adjustment Disorder: 1
Burnout: 1


In [554]:
mental_health_knowledge = [
    {
        "condition": "Major Depressive Disorder (MDD)",
        "symptoms": "Persistent sadness, loss of interest, sleep/appetite changes",
        "explanations": [
            "MDD is characterized by prolonged low mood affecting daily functioning",
            "Biological factors and life stressors often contribute to depressive episodes"
        ],
        "coping_mechanisms": [
            "Behavioral activation through scheduled activities",
            "Cognitive restructuring of negative thoughts",
            "Regular physical exercise (30 mins/day)"
        ],
        "actions": [
            "Consult psychiatrist for SSRI evaluation",
            "Begin CBT with licensed therapist",
            "Establish sleep hygiene routine"
        ]
    },
    {
        "condition": "Dysthymia",
        "symptoms": "Persistent depressive mood, low self-esteem, fatigue",
        "explanations": [
            "Dysthymia involves long-term but less severe symptoms compared to MDD",
            "It may coexist with major depressive episodes (double depression)"
        ],
        "coping_mechanisms": [
            "Psychotherapy focusing on restructuring negative thoughts",
            "Antidepressant medications in some cases",
            "Mindfulness-based stress reduction techniques"
        ],
        "actions": [
            "Consult psychologist for ongoing therapy",
            "Explore group therapy options",
            "Educate on symptom management strategies"
        ]
    },
    {
        "condition": "Seasonal Affective Disorder (SAD)",
        "symptoms": "Depression during specific seasons, low energy, oversleeping",
        "explanations": [
            "SAD is linked to reduced sunlight exposure in winter months",
            "Light therapy and lifestyle adjustments can alleviate symptoms"
        ],
        "coping_mechanisms": [
            "Light therapy with a bright light box",
            "Increase outdoor activities during daylight hours",
            "Vitamin D supplementation under medical guidance"
        ],
        "actions": [
            "Consult psychiatrist for diagnosis and treatment options",
            "Initiate light therapy regimen",
            "Monitor symptoms across seasons"
        ]
    },
    {
        "condition": "Generalized Anxiety Disorder (GAD)",
        "symptoms": "Excessive worry, restlessness, difficulty concentrating",
        "explanations": [
            "GAD involves persistent and excessive anxiety that interferes with daily life",
            "It often coexists with other anxiety disorders or depression"
        ],
        "coping_mechanisms": [
            "Cognitive-behavioral therapy (CBT) to address worry patterns",
            "Mindfulness meditation and relaxation techniques",
            "Gradual exposure therapy for specific fears"
        ],
        "actions": [
            "Consult psychologist for CBT sessions",
            "Explore medication options (e.g., SSRIs, benzodiazepines)",
            "Develop personalized coping strategies"
        ]
    },
    {
        "condition": "Panic Disorder",
        "symptoms": "Sudden panic attacks, fear of losing control, physical symptoms",
        "explanations": [
            "Panic disorder involves recurrent and unexpected panic attacks",
            "It often leads to avoidance behaviors and anticipatory anxiety"
        ],
        "coping_mechanisms": [
            "Deep breathing exercises during panic attacks",
            "Systematic desensitization to triggers",
            "Psychodynamic therapy to explore underlying causes"
        ],
        "actions": [
            "Consult psychiatrist for medication management",
            "Begin exposure therapy with a trained therapist",
            "Develop personalized panic attack response plan"
        ]
    },
    {
        "condition": "PTSD",
        "symptoms": "Intrusive memories, hypervigilance, emotional numbness",
        "explanations": [
            "PTSD results from exposure to traumatic events",
            "It can persist long after the traumatic experience has ended"
        ],
        "coping_mechanisms": [
            "Trauma-focused cognitive-behavioral therapy (CBT)",
            "Eye movement desensitization and reprocessing (EMDR)",
            "Medications targeting symptoms like nightmares or hyperarousal"
        ],
        "actions": [
            "Consult psychologist specializing in trauma therapy",
            "Initiate trauma-focused therapy sessions",
            "Join support groups for trauma survivors"
        ]
    },
    {
        "condition": "Insomnia",
        "symptoms": "Difficulty falling asleep, waking up frequently, daytime fatigue",
        "explanations": [
            "Insomnia involves persistent difficulty initiating or maintaining sleep",
            "It can result from stress, poor sleep habits, or medical conditions"
        ],
        "coping_mechanisms": [
            "Cognitive behavioral therapy for insomnia (CBT-I)",
            "Sleep hygiene education and practices",
            "Prescription medications for short-term relief"
        ],
        "actions": [
            "Consult sleep specialist for comprehensive evaluation",
            "Enroll in CBT-I program with a licensed therapist",
            "Implement sleep hygiene recommendations"
        ]
    },
    # {
    #     "condition": "Hypersomnia",
    #     "symptoms": "Excessive daytime sleepiness, prolonged sleep duration, difficulty waking up",
    #     "explanations": [
    #         "Hypersomnia refers to excessive sleepiness during the day, even after a full night’s sleep",
    #         "It can be caused by sleep disorders, depression, or certain medications"
    #     ],
    #     "coping_mechanisms": [
    #         "Maintaining a consistent sleep schedule",
    #         "Engaging in light physical activity during the day",
    #         "Avoiding caffeine or alcohol before bedtime"
    #     ],
    #     "actions": [
    #         "Consult a sleep specialist for a sleep study",
    #         "Evaluate underlying conditions like sleep apnea or narcolepsy",
    #         "Consider medication management for daytime sleepiness"
    #     ]
    # },
    {
        "condition": "Sleep Apnea",
        "symptoms": "Loud snoring, choking or gasping during sleep, daytime fatigue, trouble concentrating",
        "explanations": [
            "Sleep apnea is a condition where the airway becomes blocked during sleep, causing breathing interruptions",
            "It leads to poor sleep quality and can contribute to cardiovascular issues"
        ],
        "coping_mechanisms": [
            "Use of a CPAP (Continuous Positive Airway Pressure) machine",
            "Weight loss to reduce throat obstruction",
            "Sleeping on one's side to keep the airway open"
        ],
        "actions": [
            "Consult an ENT specialist or sleep specialist for diagnosis",
            "Consider lifestyle changes such as weight management",
            "Follow-up with regular sleep studies to monitor condition"
        ]
    },
    {
        "condition": "Suicidal Ideation",
        "symptoms": "Thoughts of death, hopelessness, feeling trapped, withdrawal from social interaction",
        "explanations": [
            "Suicidal ideation refers to thinking about or planning suicide",
            "It can stem from underlying mental health issues such as depression, trauma, or stress"
        ],
        "coping_mechanisms": [
            "Engaging in therapy (CBT or DBT) to address thought patterns",
            "Developing a crisis plan with a trusted individual",
            "Focusing on positive coping strategies such as journaling or creative expression"
        ],
        "actions": [
            "Immediate consultation with a mental health professional",
            "Emergency services or helplines for immediate safety",
            "Engage in safety measures and crisis management"
        ]
    },
    # {
    #     "condition": "High-Risk Crisis",
    #     "symptoms": "Severe distress, potential harm to self or others, disorientation, erratic behavior",
    #     "explanations": [
    #         "A high-risk crisis involves situations where there is an immediate threat to a person’s safety",
    #         "It could be a result of severe emotional distress, trauma, or psychotic episodes"
    #     ],
    #     "coping_mechanisms": [
    #         "Crisis intervention strategies to stabilize the situation",
    #         "Creating a safe environment with support from professionals",
    #         "Use of emergency psychiatric interventions if necessary"
    #     ],
    #     "actions": [
    #         "Call emergency services (911 or local crisis hotline)",
    #         "Ensure the person is in a safe space away from harm",
    #         "Engage a psychiatrist or counselor for immediate care"
    #     ]
    # },
    {
        "condition": "Chronic Stress",
        "symptoms": "Constant worry, irritability, difficulty sleeping, muscle tension, feeling overwhelmed",
        "explanations": [
            "Chronic stress occurs when an individual faces prolonged or constant stress, often without adequate relief",
            "It can contribute to a variety of physical and mental health issues, including cardiovascular problems and anxiety"
        ],
        "coping_mechanisms": [
            "Relaxation techniques such as deep breathing or progressive muscle relaxation",
            "Mindfulness meditation or yoga to reduce stress levels",
            "Engaging in regular physical exercise to release built-up tension"
        ],
        "actions": [
            "Consult with a therapist for stress management techniques",
            "Explore lifestyle adjustments like time management and work-life balance",
            "Consider medication management for stress-related symptoms"
        ]
    },
    {
        "condition": "Adjustment Disorder",
        "symptoms": "Stress-related emotional disturbances, anxiety, depression",
        "explanations": [
            "Adjustment disorder occurs in response to significant life changes or stressors",
            "It typically resolves once the stressor has passed or coping mechanisms are in place"
        ],
        "coping_mechanisms": [
            "Supportive therapy to navigate transitions",
            "Stress management techniques (e.g., relaxation exercises)",
            "Social support from friends and family"
        ],
        "actions": [
            "Consult psychologist for supportive counseling",
            "Explore stress reduction techniques",
            "Develop adaptive coping strategies"
        ]
    },
    {
        "condition": "Burnout",
        "symptoms": "Emotional exhaustion, reduced productivity, cynicism",
        "explanations": [
            "Burnout is related to chronic workplace stress and overwhelming responsibilities",
            "It can lead to physical and emotional depletion without adequate rest or recovery"
        ],
        "coping_mechanisms": [
            "Work-life balance adjustments",
            "Mindfulness practices to manage stress",
            "Career coaching or changes to reduce job strain"
        ],
        "actions": [
            "Consult occupational therapist for burnout assessment",
            "Implement stress management techniques at work",
            "Consider temporary leave or job reassignment if necessary"
        ]
    }
]

In [555]:
training_prompts = []
for condition in mental_health_knowledge:
    prompt = f"""Explain {condition['condition']} with coping strategies. Symptoms: {', '.join(condition['symptoms'])}"""
    response = f"""
    Clinical Explanation: {condition['explanations'][0]}
    Coping Strategies: {', '.join(condition['coping_mechanisms'])}
    Recommended Actions: {', '.join(condition['actions'])}
    """
    training_prompts.append({"input": prompt, "target": response})

In [556]:
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments

In [557]:
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [558]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./mentalhealth-t5",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    predict_with_generate=True,
    eval_strategy="steps",  # Update this to `eval_strategy` instead of `evaluation_strategy`
    eval_steps=500,
    save_steps=1000,
    warmup_steps=100,
    learning_rate=3e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="tensorboard"
)

In [559]:
class MentalHealthAssistant:
    def __init__(self):
        self.model = T5ForConditionalGeneration.from_pretrained("./mentalhealth-t5")
        self.tokenizer = T5Tokenizer.from_pretrained("./mentalhealth-t5")

        # # Thresholds for urgency classification (adjust as needed)
        # self.thresholds = {
        #     "Major Depressive Disorder (MDD)": 0.,
        #     "Dysthymia": 0.5,
        #     "Seasonal Affective Disorder (SAD)": 0.5,
        #     "Generalized Anxiety Disorder (GAD)": 0.55,
        #     "Panic Disorder": 0.5,
        #     "PTSD": 0.6,
        #     "Insomnia": 0.45,
        #     "Hypersomnia": 0.4,
        #     "Sleep Apnea": 0.4,
        #     "Suicidal Ideation": 0.35,
        #     "High-Risk Crisis": 0.4,
        #     "Chronic Stress": 0.5,
        #     "Adjustment Disorder": 0.45,
        #     "Burnout": 0.4
        # }

        # Thresholds for urgency classification (adjust as needed)
        self.thresholds = {
            "Major Depressive Disorder (MDD)": 0.8,
            "Dysthymia": 0.8,
            "Seasonal Affective Disorder (SAD)": 0.85,
            "Generalized Anxiety Disorder (GAD)": 0.855,
            "Panic Disorder": 0.85,
            "PTSD": 0.86,
            "Insomnia": 0.845,
            "Hypersomnia": 0.84,
            "Sleep Apnea": 0.84,
            "Suicidal Ideation": 0.835,
            "High-Risk Crisis": 0.84,
            "Chronic Stress": 0.85,
            "Adjustment Disorder": 0.845,
            "Burnout": 0.84
        }

    def generate_insights(self, condition, probability):
        """Generate a detailed explanation based on condition and probability."""
        inputs = self.tokenizer(
            f"Explain {condition} with probability {probability:.2f}",
            return_tensors="pt",
            max_length=128,
            truncation=True
        )
        
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=256,
            num_beams=4,
            early_stopping=True
        )
        
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _classify_severity(self, probability):
        """Classifies severity based on probability."""
        if probability >= 0.7:
            return "Severe"
        elif probability >= 0.5:
            return "Moderate"
        else:
            return "Mild"

    def _determine_urgency(self, condition, probability):
        """Determines urgency level for intervention based on thresholds."""
        threshold = self.thresholds.get(condition, 0.8)  # Default threshold if not specified
        if probability >= threshold:
            return "Immediate medical intervention recommended"
        elif probability >= (threshold * 0.8):  # Adjust for close monitoring cases
            return "Monitor closely, therapy required"
        else:
            return "Routine psychological support suggested"

    def format_report(self, conditions, predictions, probabilities):
        report = []
        for cond, pred, prob in zip(conditions, predictions, probabilities):
            if pred == 1:
                insight = self.generate_insights(cond, prob)
                report.append({
                    "Condition": cond,
                    "Severity": self._classify_severity(prob),
                    "Clinical Insight": insight,
                    "Urgency": self._determine_urgency(cond, prob)
                })
        return pd.DataFrame(report)


        

In [560]:
class SafetyProtocols:
    crisis_resources = {
        "US": "988 Suicide & Crisis Lifeline",
        "UK": "116 123 (Samaritans)",
        "IN": "9152987821 (Vandrevala Foundation)"
    }

    def check_emergency(self, conditions, predictions, probabilities):
        """Checks if an emergency protocol should be triggered."""
        emergency_cases = []
        
        for cond, pred, prob in zip(conditions, predictions, probabilities):
            if pred == 1 and prob >= 0.9:
                emergency_cases.append(cond["condition"])

        if emergency_cases:
            return self._emergency_protocol(emergency_cases)
        return "No immediate emergency detected."

    def _emergency_protocol(self, conditions):
        """Returns emergency response details if a crisis is detected."""
        return f"""
        [CRITICAL ALERT] Immediate Action Required:
        - High-risk conditions detected: {', '.join(conditions)}
        - Contact crisis hotlines based on location: {self.crisis_resources}
        - Ensure immediate psychological or medical intervention
        - Remove access to potential self-harm means
        - Seek emergency support from a crisis response team
        """


In [561]:
# Example usage:
mental_health_assistant = MentalHealthAssistant()
safety_protocols = SafetyProtocols()

# Generate report
report = mental_health_assistant.format_report(conditions, predictions, probabilities)
print(report)

# Check emergency status
emergency_status = safety_protocols.check_emergency(conditions, predictions, probabilities)
print(emergency_status)

In [562]:
from fpdf import FPDF

# Initialize MentalHealthAssistant and SafetyProtocols
mental_health_assistant = MentalHealthAssistant()
safety_protocols = SafetyProtocols()

# Generate report
report = mental_health_assistant.format_report(conditions, predictions, probabilities)

# Check emergency status
emergency_status = safety_protocols.check_emergency(conditions, predictions, probabilities)

# Create a PDF document to store the results
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Set title
pdf.set_font('Arial', 'B', 16)
pdf.cell(200, 10, txt="Mental Health Report", ln=True, align='C')

# Add report details
pdf.set_font('Arial', '', 12)
pdf.ln(10)  # Line break

# Write the formatted report (as a simple example)
pdf.cell(200, 10, txt="Generated Report:", ln=True)
for index, row in report.iterrows():
    condition = row['Condition']
    severity = row['Severity']
    insight = row['Clinical Insight']
    urgency = row['Urgency']
    
    # Try to find the mental health knowledge for the current condition
    condition_info = next((item for item in mental_health_knowledge if item["condition"] == condition), None)
    
    if condition_info:
        pdf.cell(200, 10, txt=f"Condition: {condition}", ln=True)
        pdf.cell(200, 10, txt=f"Severity: {severity}", ln=True)
        pdf.cell(200, 10, txt=f"Clinical Insight: {insight}", ln=True)
        pdf.cell(200, 10, txt=f"Urgency: {urgency}", ln=True)

        # Add the symptom, explanation, coping mechanisms, and actions
        pdf.ln(5)
        pdf.cell(200, 10, txt=f"Symptoms: {condition_info['symptoms']}", ln=True)
        pdf.multi_cell(200, 10, txt=f"Explanations: {'; '.join(condition_info['explanations'])}")
        pdf.multi_cell(200, 10, txt=f"Coping Mechanisms: {'; '.join(condition_info['coping_mechanisms'])}")
        pdf.multi_cell(200, 10, txt=f"Actions: {'; '.join(condition_info['actions'])}")
    else:
        # If no match is found, indicate that information is missing
        pdf.cell(200, 10, txt=f"Condition: {condition} (No additional information found)", ln=True)

    # Add a line break after each condition
    pdf.ln(5)

# Add emergency status section
pdf.ln(10)
pdf.cell(200, 10, txt="Emergency Status:", ln=True)
pdf.multi_cell(200, 10, txt=emergency_status)

# Save the PDF to a file
pdf_output_path = "mental_health_report.pdf"
pdf.output(pdf_output_path)

print(f"PDF report saved as {pdf_output_path}")


PDF report saved as mental_health_report.pdf


: 